In [44]:
import pandas as pd
import numpy as np
import os
import ta

# Constants
DATA_DIR = "../data"
TICKERS = ['SPY', 'QQQ', 'IWM']
SAVE_PREFIX = "features_"

def compute_features(df):
    df = df.copy()

    # Fix stringified tuple column names
    df.columns = [col.replace("('", "").replace("')", "").replace("', '", "_") for col in df.columns]

    # Identify the Date column
    date_col = next((col for col in df.columns if col.lower().startswith("date")), None)
    if date_col is None:
        raise KeyError(f"Could not locate date column. Got columns: {df.columns.tolist()}")

    df[date_col] = pd.to_datetime(df[date_col])
    df.set_index(date_col, inplace=True)
    df.sort_index(inplace=True)

    # Use appropriate columns
    price_col = next((c for c in df.columns if "close" in c.lower()), None)
    high_col = next((c for c in df.columns if "high" in c.lower()), None)
    low_col = next((c for c in df.columns if "low" in c.lower()), None)

    # Core returns & volatility
    df['daily_return'] = df[price_col].pct_change()
    df['rolling_vol_5'] = df['daily_return'].rolling(window=5).std()
    df['momentum_10'] = df[price_col] - df[price_col].shift(10)
    df['ret_1'] = df['daily_return'].shift(1)
    df['ret_3_mean'] = df['daily_return'].rolling(window=3).mean()

    # RSI
    df['rsi_14'] = ta.momentum.RSIIndicator(df[price_col], window=14).rsi()
    df['rsi_z'] = (df['rsi_14'] - df['rsi_14'].rolling(20).mean()) / df['rsi_14'].rolling(20).std()

    # Bollinger Band Width
    bb = ta.volatility.BollingerBands(close=df[price_col], window=20, window_dev=2)
    df['bb_width'] = bb.bollinger_hband() - bb.bollinger_lband()

    # MACD
    macd = ta.trend.MACD(close=df[price_col])
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    df['macd_hist'] = df['macd'] - df['macd_signal']

    # Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(high=df[high_col], low=df[low_col], close=df[price_col])
    df['stoch_k'] = stoch.stoch()
    df['stoch_d'] = stoch.stoch_signal()

    # ATR
    atr = ta.volatility.AverageTrueRange(high=df[high_col], low=df[low_col], close=df[price_col])
    df['atr_14'] = atr.average_true_range()

    # Optional removed: df['sma_10'], df['sma_20'], df['price_above_sma_10']

    # Target variable
    df['target'] = (df['daily_return'].shift(-1) > 0).astype(int)

    # Final cleanup
    df.dropna(inplace=True)
    df.reset_index(inplace=True)

    return df


# Process each ticker
for ticker in TICKERS:
    input_path = os.path.join(DATA_DIR, f"{ticker}.parquet")
    df = pd.read_parquet(input_path)

    print(f"🔧 Processing {ticker}")
    features_df = compute_features(df)

    output_path = os.path.join(DATA_DIR, f"{SAVE_PREFIX}{ticker}.parquet")
    features_df.to_parquet(output_path, index=False)
    print(f"✅ Saved to {output_path}\n")

print("🎉 All tickers processed.")


🔧 Processing SPY
✅ Saved to ../data/features_SPY.parquet

🔧 Processing QQQ
✅ Saved to ../data/features_QQQ.parquet

🔧 Processing IWM
✅ Saved to ../data/features_IWM.parquet

🎉 All tickers processed.
